# Основные операторы PostgreSQL.

## Цель
Научиться работать с основными операторами PostgreSQL, фильтровать таблицы по разным условиям, писать вложенные запросы, объединять таблицы.

## Формулировка задания

Дано два csv-файла с данными о клиентах (customer.csv) и их транзакциях (transaction.csv).
Необходимо выполнить следующее:

    Создать таблицы со следующими структурами и загрузить данные из csv-файлов. Детали приведены ниже.
    Выполнить следующие запросы:
        (1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.
        (1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.
        (1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.
        (1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services
        (1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.
        (1 балл) Вывести всех клиентов, у которых нет транзакций.
        (2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.
        (2 балла) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.


In [14]:
import secretstorage as sec

def get_database_password(connection: sec.DBusConnection) -> str:
    collection = sec.get_default_collection(connection)
    # Поиск секрета по атрибуту "service"
    for item in collection.search_items({'service': 'ipynb'}):
        if item.is_locked():
            item.unlock()
        return item.get_secret().decode("cp1251").strip()

    return ""


In [15]:
connection = sec.dbus_init()

In [16]:
import pandas as pd
import psycopg2 as pg

# Connect to database
db_conn = pg.connect(dbname="study", host="localhost", user="postgres", password=get_database_password(connection))

### Задание 1

(1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

In [17]:
all_brands_query = "SELECT DISTINCT brand FROM hw_2.transaction WHERE standard_cost > 1500"
pd.read_sql_query(all_brands_query, db_conn)

/tmp/ipykernel_148401/1830605956.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(all_brands_query, db_conn)


,brand
0,OHM Cycles
1,WeareA2B
2,Trek Bicycles
3,Solex
4,Norco Bicycles
5,Giant Bicycles


### Задание 2

(1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

In [18]:
approved_transactions = """
    SELECT * FROM hw_2.transaction WHERE TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-04-01' AND '2017-04-09' AND order_status = 'Approved'
"""
pd.read_sql_query(approved_transactions, db_conn)

/tmp/ipykernel_148401/2232607706.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(approved_transactions, db_conn)


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,17,79,2426,03.04.2017,False,Approved,Norco Bicycles,Standard,medium,medium,155558.0,81801.0
1,19,54,2268,06.04.2017,True,Approved,WeareA2B,Standard,medium,medium,129284.0,1344.0
2,23,37,2001,08.04.2017,True,Approved,OHM Cycles,Standard,low,medium,179343.0,24882.0
3,83,0,3398,01.04.2017,True,Approved,OHM Cycles,Standard,medium,medium,23563.0,12507.0
4,89,0,2682,04.04.2017,True,Approved,OHM Cycles,Road,high,large,1201.0,721.0
...,...,...,...,...,...,...,...,...,...,...,...,...
526,19655,0,336,09.04.2017,True,Approved,Norco Bicycles,Standard,medium,medium,3604.0,27030.0
527,19853,7,3072,02.04.2017,False,Approved,Trek Bicycles,Road,low,medium,98037.0,23443.0
528,19899,57,325,06.04.2017,False,Approved,WeareA2B,Touring,medium,large,189039.0,26014.0
529,19968,0,2751,06.04.2017,False,Approved,WeareA2B,Standard,medium,medium,6034.0,4526.0


### Задание 3

(1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [19]:
it_custimers = """
    SELECT * FROM hw_2.customer WHERE job_title LIKE 'Senior%'
"""
pd.read_sql_query(it_custimers, db_conn)

/tmp/ipykernel_148401/1000017481.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(it_custimers, db_conn)


,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,n/a,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
1,10,Fiorenze,Birdall,Female,1988-10-11,Senior Quality Engineer,Financial Services,Mass Customer,N,Yes,44339 Golden Leaf Alley,4557,QLD,Australia,4
2,28,Fee,Zellmer,Male,1973-09-30,Senior Quality Engineer,Health,Affluent Customer,N,Yes,2951 Petterle Place,2756,New South Wales,Australia,9
3,38,Cordi,Merman,Female,1955-10-29,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,1 Claremont Park,3931,VIC,Australia,10
4,45,Trace,Woodhead,Male,1975-09-23,Senior Financial Analyst,Financial Services,Mass Customer,N,No,3107 Calypso Terrace,2210,New South Wales,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3843,Mahmoud,Ligerton,Male,1989-06-06,Senior Financial Analyst,Financial Services,Mass Customer,N,Yes,38473 Waubesa Alley,2759,NSW,Australia,8
174,3851,Fowler,Bauduccio,Male,1994-11-30,Senior Sales Associate,Manufacturing,High Net Worth,N,No,20370 Fordem Pass,4825,QLD,Australia,4
175,3932,Hadley,Veracruysse,Male,1986-07-10,Senior Editor,Financial Services,Mass Customer,N,No,67473 Nova Avenue,3089,VIC,Australia,10
176,3950,Alic,Juniper,Male,1968-09-07,Senior Financial Analyst,Financial Services,High Net Worth,N,Yes,6031 Susan Terrace,4118,QLD,Australia,5


### Задание 4

(1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

In [20]:
all_brends_by_fin_service = """
    SELECT DISTINCT transaction.brand
    FROM hw_2.transaction
    WHERE customer_id IN (
        SELECT customer_id FROM hw_2.customer WHERE job_industry_category = 'Financial Services'
    )
"""
pd.read_sql_query(all_brends_by_fin_service, db_conn)

/tmp/ipykernel_148401/3391036024.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(all_brends_by_fin_service, db_conn)


,brand
0,
1,Trek Bicycles
2,WeareA2B
3,Solex
4,Giant Bicycles
5,OHM Cycles
6,Norco Bicycles


### Задание 5

(1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [21]:
top_10_clients = """
    SELECT *
    FROM hw_2.customer as c
    LEFT JOIN hw_2.transaction as t
    ON t.customer_id = c.customer_id
    WHERE t.brand = ANY(ARRAY['Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles']) LIMIT 10
"""
pd.read_sql_query(top_10_clients, db_conn)

/tmp/ipykernel_148401/3123314816.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(top_10_clients, db_conn)


,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,3120,Lauree,O'Donnell,Female,1979-02-04,Clinical Specialist,Health,Mass Customer,N,Yes,...,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,209147.0,38892.0
1,3135,Titus,Worsall,Male,1962-01-14,Staff Scientist,Financial Services,Mass Customer,N,No,...,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,119846.0,38110.0
2,787,Norma,Batrim,Female,1996-12-15,,Retail,Affluent Customer,N,Yes,...,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,17653.0,70948.0
3,2339,Damien,Haddeston,Male,1958-12-28,Web Designer II,Property,Affluent Customer,N,Yes,...,2339,08.03.2017,True,Approved,Giant Bicycles,Road,medium,medium,153899.0,82965.0
4,1986,Melania,Ruske,Female,1981-12-30,,Retail,Mass Customer,N,No,...,1986,17.01.2017,False,Approved,Trek Bicycles,Mountain,low,medium,57464.0,45971.0
5,1243,Robbert,Blakey,Male,1953-08-09,Research Assistant II,Retail,Mass Customer,N,No,...,1243,26.02.2017,True,Approved,Trek Bicycles,Standard,low,medium,105751.0,15440.0
6,2717,Frederic,McGown,Male,1981-03-25,Registered Nurse,Health,Mass Customer,N,No,...,2717,10.09.2017,False,Approved,Norco Bicycles,Standard,high,small,166192.0,147911.0
7,247,Kristal,Joysey,Female,1983-05-06,Staff Accountant II,Financial Services,High Net Worth,N,Yes,...,247,11.06.2017,False,Approved,Giant Bicycles,Standard,medium,large,17653.0,70948.0
8,2961,Merv,Barradell,Male,1975-03-22,Assistant Manager,Health,Mass Customer,N,Yes,...,2961,10.10.2017,False,Approved,Trek Bicycles,Standard,medium,large,209147.0,38892.0
9,2426,Yancy,Ovett,U,,,IT,Affluent Customer,N,Yes,...,2426,03.04.2017,False,Approved,Norco Bicycles,Standard,medium,medium,155558.0,81801.0


### Задание 6

Вывести всех клиентов, у которых нет транзакций.

In [22]:
customer_no_transaction = """
    SELECT *
    FROM hw_2.customer
    WHERE customer_id NOT IN (
        SELECT customer_id FROM hw_2.transaction)
"""
pd.read_sql_query(customer_no_transaction, db_conn)

/tmp/ipykernel_148401/3616072158.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(customer_no_transaction, db_conn)


,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,852,Andie,Bonney,Female,2000-11-04,Compensation Analyst,Financial Services,Affluent Customer,N,Yes,94 Anhalt Way,3139,VIC,Australia,7
1,869,Addia,Abels,Female,1984-03-11,Account Representative I,Financial Services,High Net Worth,N,Yes,02377 Maywood Trail,2287,NSW,Australia,7
2,1373,Shaylynn,Epsley,Female,1958-09-23,Director of Sales,Financial Services,Mass Customer,N,Yes,0 Grasskamp Pass,3170,VIC,Australia,10
3,2074,Roslyn,Rawdall,Female,1997-06-11,,Financial Services,Mass Customer,N,No,95483 Washington Junction,2505,NSW,Australia,9
4,2660,Hunt,Scollard,Male,1963-11-15,,Retail,High Net Worth,N,Yes,359 Briar Crest Road,2155,NSW,Australia,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,3996,Rosalia,Halgarth,Female,1975-08-09,VP Product Management,Health,Mass Customer,N,No,57042 Village Green Point,4511,QLD,Australia,6
503,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
504,3998,Sarene,Woolley,U,,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
505,3999,Patrizius,,Male,1973-10-24,,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7


### Задание 7

(2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

In [23]:
customers_max_price = """
    SELECT *
    FROM hw_2.customer as c
    LEFT JOIN hw_2.transaction as t
    ON t.customer_id = c.customer_id
    WHERE t.standard_cost = (
        SELECT MAX(standard_cost) FROM hw_2.transaction
    )
"""
pd.read_sql_query(customers_max_price, db_conn)

/tmp/ipykernel_148401/3073033769.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(customers_max_price, db_conn)


,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,3473,Sanderson,Alloway,U,,Analog Circuit Design manager,IT,Mass Customer,N,No,...,3473,18.12.2017,False,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
1,2135,Teador,Laurant,Male,1980-09-18,Structural Engineer,n/a,Mass Customer,N,Yes,...,2135,26.01.2017,True,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
2,3380,Abe,Ealam,Male,1955-01-09,Speech Pathologist,Manufacturing,High Net Worth,N,Yes,...,3380,28.10.2017,False,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
3,720,Darrel,Canet,Male,1931-10-23,Recruiting Manager,Retail,Affluent Customer,N,No,...,720,09.09.2017,False,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
4,1535,Emanuel,Baukham,Male,1997-04-08,Cost Accountant,Financial Services,Affluent Customer,N,Yes,...,1535,13.08.2017,False,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,3411,Alena,Hannis,Female,1980-01-04,Physical Therapy Assistant,n/a,Mass Customer,N,Yes,...,3411,23.06.2017,False,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
191,844,Tome,Tyt,Male,1978-01-13,Nurse Practicioner,Financial Services,High Net Worth,N,No,...,844,09.08.2017,False,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
192,54,Loralyn,Wonfor,Femal,1966-07-24,Sales Associate,Property,Mass Customer,N,Yes,...,54,24.10.2017,False,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0
193,1841,Nathanial,D'Ambrogi,Male,1995-10-22,Administrative Assistant III,Argiculture,Affluent Customer,N,No,...,1841,03.01.2017,True,Approved,Giant Bicycles,Standard,high,small,197736.0,175985.0


### Задание 8

(2 балла) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [24]:
customers_it_health = """
    SELECT *
    FROM hw_2.customer as c
    LEFT JOIN hw_2.transaction as t
    ON t.customer_id = c.customer_id
    WHERE TO_DATE(t.transaction_date, 'DD.MM.YYYY') BETWEEN '2017-07-07' AND '2017-07-17' AND order_status = 'Approved'
    AND c.job_industry_category = ANY(ARRAY['IT', 'Health'])
"""
pd.read_sql_query(customers_it_health, db_conn)

/tmp/ipykernel_148401/1702853049.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(customers_it_health, db_conn)


,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,2000,Kain,Rene,U,,Assistant Professor,IT,High Net Worth,N,Yes,...,2000,08.07.2017,False,Approved,Trek Bicycles,Standard,medium,medium,49953.0,38872.0
1,2337,Odilia,Bick,Female,2000-01-11,Programmer Analyst II,Health,Mass Customer,N,No,...,2337,11.07.2017,False,Approved,Trek Bicycles,Standard,medium,medium,49953.0,38872.0
2,893,Gibby,Fearnley,Male,1983-09-11,Geologist I,IT,Mass Customer,N,No,...,893,09.07.2017,False,Approved,Solex,Standard,low,medium,128985.0,7451.0
3,3166,Bronson,Plowman,Male,1955-05-11,Social Worker,Health,Mass Customer,N,No,...,3166,14.07.2017,True,Approved,WeareA2B,Road,low,small,117278.0,104377.0
4,1555,Marcello,Spenclay,Male,1962-08-19,Tax Accountant,Health,High Net Worth,N,No,...,1555,08.07.2017,False,Approved,Giant Bicycles,Standard,medium,large,16353.0,99366.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1174,Shellysheldon,Gooderridge,U,,Executive Secretary,IT,Mass Customer,N,No,...,1174,07.07.2017,False,Approved,Norco Bicycles,Standard,medium,medium,3604.0,27030.0
120,1044,Frederico,Whilder,U,,Food Chemist,Health,High Net Worth,N,No,...,1044,09.07.2017,True,Approved,WeareA2B,Standard,medium,small,141501.0,125936.0
121,1862,Daron,Umpleby,Female,1980-06-29,,Health,High Net Worth,N,Yes,...,1862,11.07.2017,True,Approved,Solex,Standard,medium,medium,44149.0,8499.0
122,712,Norine,Antonik,Female,1961-05-20,,IT,Mass Customer,N,No,...,712,15.07.2017,False,Approved,WeareA2B,Standard,medium,medium,122807.0,40091.0


In [25]:
# Close connection to the database.
db_conn.close()